## Initialisation

### Imports

In [19]:
from langchain_community.document_loaders.text import TextLoader
from langchain.embeddings.ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import numpy as np
import pickle

### Basic functions and parameters

In [20]:
def pretty_print_docs(docs):
    if isinstance(docs[0], tuple):
        print(
            f"\n{'-' * 100}\n".join(
                [
                    f"Document {i+1}: ({d[1]})\n\n{d[0].page_content}\nMetadata: {d[0].metadata}"
                    for i, d in enumerate(docs)
                ]
            )
        )
    else:
        print(
            f"\n{'-' * 100}\n".join(
                [
                    f"Document {i+1}:\n\n{d.page_content}\nMetadata: {d.metadata}"
                    for i, d in enumerate(docs)
                ]
            )
        )

def score_normalizer(val: float) -> float:
    return 1 - (1 / (1 + np.exp(val)))

def save_object(obj, filename):
    with open(filename, 'wb') as output:
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

### Document split and embedding

In [21]:
document_names = "nous_sommes_en_guerre.txt", "jeux_olympiques_paris.txt", "pacte_novation_linkedin.txt"

loaders = [TextLoader(document_name, encoding="utf-8") for document_name in document_names]

documents = []
for loader in loaders:
    documents.extend(loader.load())

embedding = OllamaEmbeddings(model="mistral")


## Vectorstore

In [22]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

docs = text_splitter.split_documents(documents)

for idx, text in enumerate(docs):
    text.metadata["id"] = idx

### FAISS

In [ ]:
from langchain_community.vectorstores.faiss import FAISS, DistanceStrategy

In [ ]:
vectordb = FAISS.from_documents(docs, embedding, normalize_L2=True, distance_strategy=DistanceStrategy.EUCLIDEAN_DISTANCE)
vectordb.save_local("local_vectorstore_faiss")

In [ ]:
vectordb = FAISS.load_local("local_vectorstore_faiss", embeddings=embedding, allow_dangerous_deserialization=True, normalize_L2=True, distance_strategy=DistanceStrategy.EUCLIDEAN_DISTANCE)

In [ ]:
query = """c'est quoi la meilleure discipline ?"""
docs_proba = vectordb.similarity_search_with_score(query, k=4)
pretty_print_docs(docs_proba)

### Chroma

In [23]:
from langchain.vectorstores.chroma import Chroma

In [33]:
vectordb = Chroma.from_documents(docs, embedding, persist_directory="local_vectorstore_chroma", collection_metadata={"hnsw:space": "cosine"})

TypeError: Chroma.__init__() got an unexpected keyword argument 'distance_strategy'

In [30]:
vectordb = Chroma(embedding_function=embedding, persist_directory="local_vectorstore_chroma", collection_metadata={"hnsw:space": "cosine"})

In [31]:
query = """c'est quoi la meilleure discipline ?"""
docs_proba = vectordb.similarity_search_with_score(query, k=4)
pretty_print_docs(docs_proba)

Document 1: (134508.01542657672)

Le gouvernement, dès demain, précisera toutes ces mesures. Elles seront en fonction des besoins, des réalités économiques, des nécessités secteur par secteur, évidemment adaptées. Nous serons au rendez-vous pour que notre économie soit préservée dans cette période si dure et pour que l’ensemble des travailleuses et des travailleurs puissent avoir cette sécurité aussi en termes de pouvoir d’achat, de continuité de leur vie.
Metadata: {'id': 19, 'source': 'nous_sommes_en_guerre.txt'}
----------------------------------------------------------------------------------------------------
Document 2: (134508.01542657672)

Le gouvernement, dès demain, précisera toutes ces mesures. Elles seront en fonction des besoins, des réalités économiques, des nécessités secteur par secteur, évidemment adaptées. Nous serons au rendez-vous pour que notre économie soit préservée dans cette période si dure et pour que l’ensemble des travailleuses et des travailleurs puissent a

## Multi Vector 

In [ ]:
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.retrievers.multi_vector import SearchType
from langchain.vectorstores.chroma import Chroma
from langchain.storage import InMemoryByteStore
import uuid

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000)
docs = text_splitter.split_documents(docs)

In [ ]:
vectordb = Chroma(collection_name="full_documents", embedding_function=embedding, persist_directory="local_multivector_chroma")

# The storage layer for the parent documents
store = InMemoryByteStore()
id_key = "doc_id"

# the retriever
retriever = MultiVectorRetriever(
    vectorstore=vectordb,
    byte_store=store,
    id_key=id_key,
)

doc_ids = [str(uuid.uuid4()) for _ in docs]

In [ ]:
# The splitter to use to create smaller chunks
child_text_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

In [ ]:
sub_docs = []
for i, doc in enumerate(docs):
    _id = doc_ids[i]
    _sub_docs = child_text_splitter.split_documents([doc])
    for _doc in _sub_docs:
        _doc.metadata[id_key] = _id
    sub_docs.extend(_sub_docs)

In [ ]:
retriever.vectorstore.add_documents(sub_docs, persist_directory="local_multivector_chroma")
retriever.docstore.mset(list(zip(doc_ids, docs)))

In [ ]:
# load the retriever vectorstore
retriever.vectorstore = Chroma(collection_name="full_documents", embedding_function=embedding, persist_directory="local_multivector_chroma")
retriever.docstore.mset(list(zip(doc_ids, docs)))

In [ ]:
query = """c'est quoi le discours du président ? Emanuel Macron pense que c'est la crise ?"""
docs_proba = retriever.vectorstore.similarity_search_with_score(query, k=4)
pretty_print_docs(docs_proba)

In [ ]:
# Retriever returns larger chunks
retriever.search_type = SearchType.mmr
print("\n######\n".join(
    [d.page_content 
     for d in retriever.get_relevant_documents(
         "c'est quoi le discours du président ? Emanuel Macron pense que c'est la crise ?"
         )
    ]
))